In [ ]:
CREATE OR REPLACE FUNCTION calculate_percent_profit_loss(curr FLOAT, prev FLOAT) 
RETURNS FLOAT 
LANGUAGE SQL 
AS 
$$
    CASE 
        WHEN prev = 0 THEN 100
        ELSE trunc(((curr - prev) / prev) * 100,2)
    END
$$;